# "Title"
> "Awesome summary"

- toc: true
- badges: true
- comments: true
- categories: []
- image: images/some_folder/your_image.png

```Dockerfile
# VERSION 
# AUTHOR: Ashton Sidhu
# DESCRIPTION: Basic MlFlow Tracking Container
# BUILD: docker build --rm -t bigsidhu/docker-mlflow .
# SOURCE: 

FROM python:3.7-slim-buster
LABEL maintainer="Ashton"

# Never prompt the user for choices on installation/configuration of packages
ENV DEBIAN_FRONTEND noninteractive
ENV TERM linux

# MLFlow
ARG MLFLOW_VERSION=1.12.0
ARG MLFLOW_USER_HOME=/home/mlflow


# Define en_US.
ENV LANGUAGE en_US.UTF-8
ENV LANG en_US.UTF-8
ENV LC_ALL en_US.UTF-8
ENV LC_CTYPE en_US.UTF-8
ENV LC_MESSAGES en_US.UTF-8

RUN useradd -ms /bin/bash -d ${MLFLOW_USER_HOME} mlflow \
    && pip install mlflow==$MLFLOW_VERSION \
    && pip install psycopg2-binary \
    && chown -R mlflow: ${MLFLOW_USER_HOME}

EXPOSE 5000

COPY scripts/entrypoint.sh /entrypoint.sh

USER mlflow
WORKDIR ${MLFLOW_USER_HOME}
ENTRYPOINT ["/entrypoint.sh"]
```

```yaml
version: '3.7'
services:
    postgres:
        image: postgres:latest
        environment:
            - POSTGRES_USER=mlflow
            - POSTGRES_PASSWORD=mlflow
            - POSTGRES_DB=mlflow
            - PGDATA=/var/lib/postgresql/data/pgdata
        logging:
            options:
                max-size: 10m
                max-file: "3"
        volumes:
            - ./pgdata:/var/lib/postgresql/data/pgdata

    mlflow:
        build:
            context: .
            dockerfile: Dockerfile
        image: bigsidhu/mlflow
        container_name: mlflow_server
        environment:
            - DEFAULT_ARTIFACT_ROOT=/home/mlflow/mlruns
        depends_on:
            - postgres
        restart: always
        ports:
            - "5000:5000"
        # Extra mounts, i.e for your artifact store if you are using NFS
        volumes:
            - /mnt/shared/mlruns:/home/mlflow/mlruns
```

```bash
#!/usr/bin/env bash

# User-provided configuration must always be respected.
#
# Therefore, this script must only derives MlFlow MLFLOW__ variables from other variables
# when the user did not provide their own configuration.

# Global defaults and back-compat
POSTGRES_HOST="${POSTGRES_HOST:-postgres}"
POSTGRES_PORT=5432
POSTGRES_CREDS="${POSTGRES_CREDS:-mlflow:mlflow}"
CONN_STRING="postgresql+psycopg2://${POSTGRES_CREDS}@${POSTGRES_HOST}:${POSTGRES_PORT}/mlflow${POSTGRES_EXTRAS}"

mlflow server \
    --backend-store-uri "${CONN_STRING}" \
    --default-artifact-root "${DEFAULT_ARTIFACT_ROOT}" \
    --host 0.0.0.0

```

In [1]:
try:
    import mlflow
except:
    !pip install mlflow

In [3]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse

import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
mlflow.set_tracking_uri("http://192.168.0.10:5000")

In [6]:
warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha = 0.5
l1_ratio = 0.5

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
if not mlflow.get_experiment_by_name("MLFlow Demo"):
    mlflow.create_experiment("MLFlow Demo", "/home/jovyan/shared/mlruns")

mlflow.set_experiment("MLFlow Demo")

In [12]:
with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(lr, "model")

Successfully registered model 'ElasticnetWineModel'.
2020/11/13 06:03:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetWineModel, version 1


Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


Created version '1' of model 'ElasticnetWineModel'.
